# Module 4: Measuring plant phenotypes with PlantCV - Multiple plants

[PlantCV homepage](https://plantcv.danforthcenter.org/)

[PlantCV documentation](https://plantcv.readthedocs.io/en/stable/)

In [ ]:
%matplotlib widget
import matplotlib
from plantcv import plantcv as pcv
import numpy as np
import cv2

In [ ]:
matplotlib.rcParams["figure.max_open_warning"] = False
pcv.params.debug = "plot"
pcv.params.text_size = 10
pcv.params.text_thickness = 10
pcv.params.line_thickness = 10

In [ ]:
pcv.__version__

 ## Refresher: plant segmentation

In [ ]:
# Open image file
img, imgpath, imgname = pcv.readimage(filename="images/10.9.1.244_pos-165-002-009_2020-02-29-20-05.jpg")

In [ ]:
# Convert the RGB image into a grayscale image by choosing one of the HSV or LAB channels
gray_img = pcv.rgb2gray_lab(rgb_img=img, channel="a")

In [ ]:
# Instead of setting a manual threshold, try an automatic threshold method such as Otsu
bin_img = pcv.threshold.otsu(gray_img=gray_img, max_value=255, object_type="dark")

In [ ]:
# Remove "salt" noise from the binary image
filter_bin = pcv.fill(bin_img=bin_img, size=100)

## Measuring the shape and color of objects in digital image

At this stage we have a binary mask that labels plant pixels (white) and background pixels (black). There are multiple plants but we cannot tell which pixels belong to each plant

In [ ]:
# Identify connected components (contours) using the binary image
cnt, cnt_str = pcv.find_objects(img=img, mask=filter_bin)

In [ ]:
# Plot each contour to see where they are
pcv.params.color_sequence = "random"
cp = img.copy()
for i in range(0, len(cnt)):
    cv2.drawContours(cp, cnt, i, pcv.color_palette(num=100, saved=False)[0], thickness=-1, hierarchy=cnt_str)
pcv.plot_image(cp)

We have distinct contours for each plant and some (most) plants are composed of multiple contours, how do we assign these to individual plants?

In [ ]:
# Create a region of interest (ROI) for one plant
roi, roi_str = pcv.roi.circle(img=img, x=1460, y=1400, r=100)

In [ ]:
# Filter the contours using the ROI
plant_cnt, plant_str, plant_mask, plant_area = pcv.roi_objects(img=img, roi_contour=roi, roi_hierarchy=roi_str, object_contour=cnt, obj_hierarchy=cnt_str)

In [ ]:
# Flatten contours into a single object
# Combine the contours into a single plant object
plant, mask = pcv.object_composition(img=img, contours=plant_cnt, hierarchy=plant_str)

In [ ]:
# Measure the size and shape of the plant
shape_img = pcv.analyze_object(img=img, obj=plant, mask=mask)

In [ ]:
# Output measurements
print(f"Leaf area = {pcv.outputs.observations['default']['area']['value']} pixels")
print(f"Convex hull area = {pcv.outputs.observations['default']['convex_hull_area']['value']} pixels")
print(f"Solidity = {pcv.outputs.observations['default']['solidity']['value']}")
print(f"Perimeter = {pcv.outputs.observations['default']['perimeter']['value']} pixels")
print(f"Width = {pcv.outputs.observations['default']['width']['value']} pixels")
print(f"Height = {pcv.outputs.observations['default']['height']['value']} pixels")
print(f"Center of mass = {pcv.outputs.observations['default']['center_of_mass']['value']}")
print(f"Convex hull vertices = {pcv.outputs.observations['default']['convex_hull_vertices']['value']}")
print(f"Plant in frame = {pcv.outputs.observations['default']['object_in_frame']['value']}")
print(f"Bounding ellipse center = {pcv.outputs.observations['default']['ellipse_center']['value']}")
print(f"Bounding ellipse center major axis length = {pcv.outputs.observations['default']['ellipse_major_axis']['value']} pixels")
print(f"Bounding ellipse center minor axis length = {pcv.outputs.observations['default']['ellipse_minor_axis']['value']} pixels")
print(f"Bounding ellipse angle of rotation = {pcv.outputs.observations['default']['ellipse_angle']['value']} degrees")
print(f"Bounding ellipse eccentricity = {pcv.outputs.observations['default']['ellipse_eccentricity']['value']}")

In [ ]:
# Measure the color properties of the plant
color_hist = pcv.analyze_color(rgb_img=img, mask=mask, colorspaces="hsv")

![HSV color model](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Hsl-hsv_models.svg/600px-Hsl-hsv_models.svg.png)

In [ ]:
# Output measurements
print(f"Hue circular mean = {pcv.outputs.observations['default']['hue_circular_mean']['value']} degrees")
print(f"Hue circular mean standard deviation = {pcv.outputs.observations['default']['hue_circular_std']['value']} degrees")
print(f"Hue median = {pcv.outputs.observations['default']['hue_median']['value']} degrees")